## 1.レース情報をスクレイピング

jupyter notebook で [ImportError: lxml not found, please install it]エラーが出た場合は [pip3 install lxml] でインストール後プロジェクトを再起動


In [112]:
import pandas as pd
from tqdm import tqdm
import time
import csv
import numpy as np

url >> race_id=2022 09 01 11 01 : 年  競馬場(01:札幌,02:函館,03:福島,04:新潟,05:東京,06:中山,07:中京,08:京都,09:阪神,10:小倉) 何回(今年その競馬場でのレース回数) 何日目(何レース目)

In [113]:
pd.set_option('display.max_rows',5)
pd.options.display.width = 200
pd.options.display.column_space = 100

スクレイピング

In [ ]:
# df = pd.read_pickle("1.7.df.pickle")
# already = df.index.unique()
# already = list(map(str, already))
# already

In [ ]:
nondata = []
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in already:
    #   continue
    time.sleep(1)
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      nondata.append(_race_id)
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict

dictのkeyをrace_idに変換


In [ ]:
def get_house_data():
  race_id_lists = []

  for 年 in range(2021,2022):
    for 競馬場 in range(1,3): # 1,11
      for 回 in range(1,4): # 1,11
        for 回目 in range(1,4): # 1,11
          for レース回 in range(1,13): # 1,13
            race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
            race_id_lists.append(race_id)
            
  race_result = scrape_race_results(race_id_lists)
  for key in race_result.keys():
    race_result[key].index = [key] * len(race_result[key])
    
  results = pd.concat([race_result[key] for key in race_result.keys()],sort=False)
  return results

新規でdata取得するとき

In [ ]:
house_data = get_house_data()

取得できない数値

In [ ]:
nondata

追加でdata取得するとき

In [ ]:
house_data

csvで保存

In [ ]:
house_data.to_csv('1.2.test1.csv')
# house_data.to_csv('1.2.test2.csv')

## 2.データの自動修正

1.csvデータがうまく取得できていない箇所を[サイト](https://race.netkeiba.com/race/result.html?race_id=202102010203)からコピー

2.csvファイルにペースト

3.修正箇所の行番号を"arr"に入力

In [125]:
# arr = [6,7,184,340,341,722,813,814]
arr = [6,7,238,262,319,320,323,322,73,319]
clean_index = list(map(lambda x: x-1,arr))
clean_index

[5, 6, 237, 261, 318, 319, 322, 321, 72, 318]

In [126]:
# with open('1.2.test1.csv') as f:
# or
with open('1.2.test2.csv') as f:
    reader = csv.reader(f)
    l = [row for row in reader]
    l_length = len(l)
    columns_len = len(l[0])
    print(columns_len)
    print(len(l[338]))
    print(l_length)
    
    for row in range(l_length):
      if len(l[row]) != columns_len:
        print(row)
      if row in clean_index:
        l[row] = list(map(lambda x : x.replace('\t',","),l[row]))
        text = ' '.join(l[row])
        text = text.split(",")
        text = list(map(lambda x : x.strip(),text))
        text = ' '.join(text)
        text = text.replace(' ',",")
        text = text.split(",")
        arr_len = len(text)
        for _ in range(columns_len-arr_len):
          text.append("")
        l[row] = text
        print(l[row])

22
22
463
['202101010301', '5', '5', '10', 'マニトゥーリン', '牝3', '51.0', '▲小沢', '1:08.8', '1.1/4', '12', '62.1', '34.8', '6-5', '栗東吉田', '428(+8)', '', '', '', '', '', '']
['202101010301', '6', '7', '14', 'トミケンサルタール', '牝3', '54.0', '坂井', '1:09.1', '1.1/2', '7', '29.7', '34.2', '14-9', '美浦尾形', '456(+8)', '', '', '', '', '', '']
72
['202101010306', '6', '1', '1', 'ミュージアムピース', '牡2', '54.0', '黛', '1:02.7', '2.1/2', '7', '29.1', '36.6', '7-7', '美浦伊藤大', '432(0)', '', '', '', '', '', '']
['202101020308', '1', '5', '5', 'ブーケオブアイリス', '牡3', '54.0', 'ルメー', '0:59.9', '', '1.0', '2.0', '35.3', '8-6', '美浦杉浦', '496(+2)', '', '', '', '', '', '']
['202101020310', '1', '4', '7', 'ケルンキングダム', 'セ5', '57.0', '国分恭', '2:01.3', '', '7.0', '22.6', '35.3', '8-7-7-5', '栗東小崎', '442(-6)', '', '', '', '', '', '']
318
['202102010302', '6', '5', '6', 'インディナート', '牝3', '54.0', '団野', '1:00.7', 'クビ', '2', '3.2', '36.8', '7-9', '栗東浅見', '434(0)', '', '', '', '', '', '']
['202102010302', '7', '6', '8', 'スノームーンライト', '牡3', '56.0', '

In [127]:
add = np.array(l[1:]).reshape(l_length-1,columns_len)
df_add = pd.DataFrame(add, columns=l[0])
print(df_add)

# df_add.to_csv('1.2.test1.csv',index=False)
# or
df_add.to_csv('1.2.test2.csv',index=False)

                   着順  枠  馬番         馬名  性齢    斤量    騎手     タイム     着差  ...   後3F      コーナー通過順     厩舎   馬体重(増減) Unnamed: 15 Unnamed: 16 Unnamed: 17 Unnamed: 18 Unnamed: 19 Unnamed: 20
0    202101010301   1  1   2    ミトグラフィア  牝3  53.0   ☆亀田  1:08.4         ...  34.3          6-5  美浦高柳瑞    408(0)                                                                        
1    202101010301   2  6  11     ララマカロン  牝3  54.0    大野  1:08.4     ハナ  ...  34.6          2-2  美浦田中博   466(-4)                                                                        
2    202101010301   3  2   4   カワカミカリーナ  牝3  52.0   △泉谷  1:08.6      1  ...  35.0          1-1   栗東本田   462(+4)                                                                        
3    202101010301   4  2   3      ロゼクラン  牝3  54.0    団野  1:08.6     クビ  ...  34.7          5-3  栗東高柳大  448(+10)                                                                        
4    202101010301   5  5  10    マニトゥーリン  牝3  51.0   ▲小沢  1:08.8  1.1/4  ...  34.

In [122]:
time.sleep(2)
# df = pd.read_csv("1.2.test1.csv")
# or
df = pd.read_csv("1.2.test2.csv")
# df

In [123]:
pd.set_option('display.max_rows',None)

df.columns
print(df["着順"].value_counts())
print(df["枠"].value_counts())
print(df["馬番"].value_counts())
print(df["馬名"].value_counts())
print(df["性齢"].value_counts())
print(df["斤量"].value_counts())
print(df["騎手"].value_counts())
print(df["タイム"].value_counts())
print(df["着差"].value_counts())
print(df["人気"].value_counts())
print(df["単勝オッズ"].value_counts())
print(df["後3F"].value_counts())
print(df["コーナー通過順"].value_counts())
print(df["厩舎"].value_counts())
print(df["馬体重(増減)"].value_counts())

5     37
7     37
1     36
2     36
3     36
4     36
8     36
6     35
10    32
9     32
11    30
12    27
13    21
14    14
15     8
16     5
中止     2
取消     2
Name: 着順, dtype: int64
8    69
7    68
6    66
5    64
4    57
3    53
2    44
1    41
Name: 枠, dtype: int64
2     36
4     36
3     36
1     36
6     36
8     36
7     36
5     36
9     33
10    32
11    30
12    28
13    21
14    17
15     8
16     5
Name: 馬番, dtype: int64
ラップリーダー        2
アンジェリーブル       2
ボイオートス         2
レープハフト         2
レッドクラーガ        2
コーズウェイヘッド      2
セイウンダイモス       2
ハートフルワールド      2
ワルツフォーラン       2
ラピカズマ          2
サトノディーバ        2
エンドステージ        2
メイショウシンウチ      2
シーチェンジ         2
ウインカンターレ       2
ディープリッチ        2
エルファシル         2
ミウィ            2
アダムスマイリー       2
ディキシー          2
ラヴィンジャー        2
パラマウント         2
ウインベイランダー      2
モイ             2
ディーグランデ        2
レッドエステーラ       2
ブライエス          2
クアドリフォリオ       2
ニシノリース         2
エンペラーズパレス      2
ウヒョウノヒビキ       2
クールシラユリ        2
シェーン           2
マ

In [ ]:
house_data[house_data["厩舎"] == "36.8"].index